# count-based algorithm

## word-context matrix M

$M \in \mathbb{R}^{|V_W|\times |V_C|}$ is a word-context matrix 

each row $i$ corresponds to a word $\mathbf{v}_w$

each column $j$ corresponds to a context $\mathbf{v}_c$

entry of a word-context matrix $M$ is an correlation measure between the word $\mathbf{v}_w$ and the context $\mathbf{v}_c$

$$
M_{i,j} = f(\mathbf{v}_w, \mathbf{v}_c)= \left \langle \mathbf{v}_w, \mathbf{v}_c \right \rangle = \mathbf{v}_w^T \mathbf{v}_c 
$$

where $\mathbf{v}_w \in \mathbb{R}^d$ is embedding of word $w \in V_W$

$\mathbf{v}_c \in \mathbb{R}^d$ is embedding of context word $c \in V_C$

**the entries in the vectors are latent and are the params to be learned**

$V_W$ is the word vocabulary

$V_C$ is the context vocabulary

for convenience, sometimes take $V=V_W=V_C$

$|V_W|$ is the size of the word vocabulary $V_W$ 

$|V_C|$ is the size of the context vocabulary $V_C$

size means the number of words in a vocabulary

sometimes refer $\mathbf{v}_w^T$ as rows in word embedding matrix $W \in \mathbb{R}^{|V_W|\times d}$

refer $\mathbf{v}_c^T$ as rows in context embedding matrix $C \in \mathbb{R}^{|V_C|\times d}$

then $W_i$ refers to word embedding of ith word in the word vocabulary $V_W$

$C_i$ refers to word embedding of ith context in the context vocabulary $V_C$

rows of $W$ are often used in NLP tasks, such as compute word similarities while $C$ is ignored

Note: 

- word and context can come from different vocabulary


- a context may not occur in our corpus (unobserved/randomly sampled context)


- our corpus is an observed collection of docs after segmantation and tokenization


- $D$ is a collection of observed word-context pairs, which is **a subset of our corpus**,sometimes also called corpus

- the words (observed words) in our corpus also belong to word vocabulary

$$
w_i \in V_W
$$

- and corresponding contexts in our corpus also belong to context vocabulary
    
$$
c_i \in V_C
$$

## types of word-context matrix

- PMI (pointwise mutual info matrix)


- SPMI (**shifted** pointwise mutual info matrix)


- SPPMI (shifted **positive** pointwise mutual info matrix)

### average mutual info

**average mutual info**:

$$
\begin{align}
I(w_1,w_2)

&=\sum _{w_1,w_2} p(w_1,w_2)\log\left [ \frac{p(w_1,w_2)}{p(w_1)p(w_2)} \right ] \\[1em]

&=\sum _{w_1} p(w_2|w_1)\log\left [ \frac{p(w_2|w_1)}{p(w_2)} \right ]\\[1em]

&=H(w_2)-H(w_2|w_1)\\[1em]
\end{align}
$$

where $H$ is **entropy**

$H(w_2)$ measures uncertainty of word $2$:

$$
H(w_2)=\sum _{w_2} p(w_2)\log\left [ \frac{1}{p(w_2)} \right ]
$$

### PMI (pointwise mutual information)

introduced by Church and Hanks

PMI is an info-theoretic correlation measure between a pair of dicrete outcomes $x$ and $y$

- definition:

$$
\text{PMI}(x,y) = \log \frac{P(x,y)}{P(x)P(y)}
$$


- in NLP context, $\text{PMI}(w,c)$ measures the correlation between a word $w$ and a context $c$ 

    $PMI>0$: very likely occur together
    
    $PMI<0$: not very likely occur together

    by calculating the log of the ratio between their **joint probability** (the frequency they occur together) and their **marginal probabilities**) (the frequency they occur independently)

$$
\text{PMI}(w,c) = \log \frac{P(w,c)}{P(w)P(c)}= \log \left [\frac{\frac{\#(w,c)}{|D|}}{\frac{\#(w)}{|D|}\frac{\#(c)}{|D|}}  \right ]= \log\left [\frac{\#(w,c)\cdot |D|}{\#(w) \cdot \#(c)}  \right ]
$$

where $\#(w,c)$ is the number of times the specific word-context pair $(w,c)$ occurs in $D$

$\#(w)$ is the number of times the specific word $w$ occurs in $D$

$$
\#(w) = \sum_{c' \in V_C} \#(w,c')
$$

$\#(c)$ is the number of times the specific context $c$ occurs in $D$

$$
\#(c) = \sum_{w' \in V_W} \#(w',c)
$$

dis of PMI matrix

- entry of unobserved word-context pairs $(w,c)$ is $- \infty$
      
$$
PMI(w,c) = \log\left [\frac{\#(w,c)\cdot |D|}{\#(w) \cdot \#(c)}  \right ] = \log\left [\frac{0\cdot |D|}{\#(w) \cdot \#(c)}  \right ] = \log 0 = - \infty
$$
    
- dense: a major practical issue coz PMI's huge dimension $V_W \times V_C$
        

- one smooth method:

    use **Dirichlet prior** by adding a small "fake" count to PMI can make all word-context pairs **observed**
    
    but the PMI matrix is still dense

### SPMI

- the optimal solution of SGNS is **inplicitly factorizing a shifted PMI matrix**

$$
M = W C^T
$$

- recall entry of a word-context matrix $M$ is an correlation measure between the word $\mathbf{v}_w$ and the context $\mathbf{v}_c$

$$
M_{i,j} = f(\mathbf{v}_w, \mathbf{v}_c)= \left \langle \mathbf{v}_w, \mathbf{v}_c \right \rangle = \mathbf{v}_w^T \mathbf{v}_c 
$$

**but we don't know which correlation measure $f(\mathbf{v}_w, \mathbf{v}_c)$ SGNS is using**

Thus $M$ is an **implicit** matrix, which comes the name **implicit matrix factorization**

- **local training objective** for a specific word-context pair $(w,c)$

$$
\hat{\mathbf{v}}_w, \hat {\mathbf{v}}_c =\arg \max l(w,c) = \arg \max_{\mathbf{v}_w, \mathbf{v}_c}\# (w,c)\left[ \log (\sigma (\mathbf{v}_w^T \mathbf{v}_c )) + k \cdot \# (w) \cdot \frac{\# (c)}{|D|} [\log \sigma (- \mathbf{v}_w^T v_{c_N} )] \right]
$$

derived from a sufficiently large dimensionality $d$

which allows for a perfect reconstruction of $M$

dot product of word-context pair can assume to be independent of each other

thus we can treat the global objective $l$ as a function of independent $\mathbf{v}_w^T \mathbf{v}_c$ terms


- define $x=\mathbf{v}_w^T \mathbf{v}_c$, set $\frac{\partial l}{\partial x}=0$

    the optimal local solution is 

$$
x=\mathbf{v}_w^T \mathbf{v}_c = \log \left (\frac{\#(w,c) \cdot |D| }{\#(w) \cdot \#(c)} \right )-\log k
$$

interestingly, $\log \left (\frac{\#(w,c) \cdot |D| }{\#(w) \cdot \#(c)} \right )$ is just the well-known pointwise mutual information (PMI) of $(w,c)$


then we find the matrix $M$ that SGNS is factorizing:

$$
M_{ij}=W_i C_j = v_{w_i}^T v_{c_j} = \text{PMI}(w_i,c_j ) -\log k
$$

- $k$ is hyperparamter

when $k=1$, $M$ is PMI matrix

$$
M^{\text{PMI}} = \text{PMI}(w,c)
$$

when $k>1$, $M$ is **shifted PMI matrix (SPMI)**

$$
M^{\text{PMI}_k}  = M^{\text{PMI}} -\log k = \text{PMI}(w,c) -\log k 
$$

**certain value of $k$ can improve performance on different tasks**

### SPPMI

- it's impractical to directly use very high-dimensional and dense shifted PMI matrix

- one solution: a **sparse but inconsistent** matrix $M_0^{PMI}$

    replace all the entries of **unobserved** word-context pairs ($-\infty$) with $0$

$$
PMI(w,c)=0 \text{ if } \#(w,c)=0
$$

- problem: **unobserved** word-context pairs have entries **0**

    but **observed and uncorrelated** word-context pairs have **negative** entries


- another better solution: a **sparse and consistent** matrix SPPMI

    replace all the entries with **negative values** with $0$

    thus the matrix is called **positive**

$$
\text{PPMI}(w,c)=\max(PMI(w,c), 0)
$$


- SPPMI is better at optimizing SGNS's training objective


from PPMI, when $k>1$, we derive another embedding algorithm

$$
SPPMI_k(w,c)=\max(PMI(w,c)-\log k, 0)
$$

as SGND, certain value of $k$ can improve performance on different tasks

- Intuition of discarding negative value: 

    humans can easily think of positive examples of positive association (e.g. Micky and Mouse) 

    but hard to invent negative examples of negative association (e.g., Micky and turkey). 

## NCE (noise-contrastive estimation)

similar to SGNS, NCE can also be viewed as implictly factorizing a word-contex matrix

$M$ is a shifted log-conditional-probability matrix

$$
M  = \log P(w|c) -\log k
$$

where $\log P(w|c)$ is a log-conditional-probability matrix

## SVD

### truncated SVD

truncated SVD is a basic algorithm from linear algebra to achieve optimal **rank d** factorization with respect to $l_2$ loss

SVD factorize $M \in \mathbb{R}^{|V| \times |V|}$ into product of 3 matrices

$$
M=U\Sigma V^T
$$

where $U \in \mathbb{R}^{|V| \times |V|}$ is an orthonormal matrix with columns are the left singular vectors of $M$

$\Sigma \in \mathbb{R}^{|V| \times |V|}$ is a diagonal matrix with diagonal entries of singular values of $M$

$V \in \mathbb{R}^{|V| \times |V|}$ is an orthonormal matrix with columns are the right singular vectors of $M$

recall 4th interpretation of PCA: low-rank (k) matrix approximation 

training goal: optimize over **rank d** matrix $M'$ to minimize $l_2$ reconstruction error

$$
\hat M' = \underset{\text{Rank}(M')=d}{\arg \min}=\left \| M'-M \right \|_2^2
$$

the optimal solution is the rank d matrix $M_d \in \mathbb{R}^{|V| \times |V|}$ that best approximate the original matrix $M$

$$
\hat M' =M_d=U_d\Sigma_d V_d^T
$$

where $U_d  \in \mathbb{R}^{|V| \times d}$ is an orthonormal matrix with columns are the **top d** left singular vectors of $M$

$\Sigma_d \in \mathbb{R}^{d \times d}$ is a diagonal matrix with **top d** diagonal entries of singular values of $M$

$V_d \in \mathbb{R}^{|V| \times d}$ is an orthonormal matrix with columns are the **top d** right singular vectors of $M$

thus, given $|V| \gg d$, the dense, d dimensional rows of $W=U_d \Sigma_d$ is a perfect substitues for the very high-dimensional ($|V|$) rows of $M$

$$
W^{SVD}=U_d \Sigma_d \in \mathbb{R}^{|V| \times |V|}\\[1em]
C^{SVD}=V_d \in \mathbb{R}^{|V| \times |V|}
$$


- word embedding $\mathbf{w}_i \in \mathbb{R}^{d}$:

$$
\mathbf{w}_i = [U_d \Sigma_d]_i = W_i
$$

- context embedding $\mathbf{c}_i \in \mathbb{R}^{d}$:

$$
\mathbf{c}_i = [V_d]_i = C_i
$$



### symmetric SVD

- word embedding matrix $W^{SVD}$ and context embedding matrix $C^{SVD}$ have different properties:

    $W^{SVD}$ is not orthonormal, $C^{SVD}$ is orthonormal


- while factorization achieved by SGNS is more **symmetric**

    neither $W^{W2V}$ nor $C^{W2V}$ is orthonormal

    and no bias is given to either matrics in training objective


- thus we propose another SVD algorithm: symmetric SVD

$$
W^{SVD_{1/2}}=U_d {\Sigma_d}^{1/2}\\[1em]
C^{SVD_{1/2}}=V_d {\Sigma_d}^{1/2}
$$

- this algorithm perform better than previous SVD algorithm

- this algorithm can be generalized to, making $\alpha$ a tunable parameter

$$
W^{SVD_{\alpha}}=U_d {\Sigma_d}^{\alpha}\\[1em]
C^{SVD_{\alpha}}=V_d {\Sigma_d}^{\alpha}
$$